In [1]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import statistics as stat
import seaborn as sns

In [2]:
# Starting dataset
df = pd.read_csv('cleaned_dataframe.csv', sep='\t', index_col=0)

In [ ]:
df.head()

In [ ]:
df.info()

In [3]:
# Create the dataframe for customers
data = pd.core.frame.DataFrame({'CustomerID' : df['CustomerID'].unique()})
print("Number of unique customers: ", len(data))

Number of unique customers:  4339
Number of unique customers:  4339


In [6]:
# Handle Discount

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400288 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         400288 non-null  object 
 1   BasketDate       400288 non-null  object 
 2   Sale             400288 non-null  float64
 3   CustomerID       400288 non-null  int64  
 4   CustomerCountry  400288 non-null  object 
 5   ProdID           400288 non-null  object 
 6   ProdDescr        400288 non-null  object 
 7   Qta              400288 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 27.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 400288 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         400288 non-null  object 
 1   BasketDate       400288 non-null  object 
 2   Sale             400288 non-null  float64
 3   Custome

In [34]:
# test block on discount stuff

unique_products = []
for customer in tqdm(df['CustomerID'].unique(), total=len(data), desc="Iterating over customers"):
    # use set ?
    #df_temp = (df.loc[(df['CustomerID'] == customer), ['ProdID']].drop_duplicates())
    #unique_products.append(df_temp.loc[df_temp['ProdID'] != 'D'].to_numpy())
    #unique_products.append(df.loc[(df['CustomerID'] == customer) & (df['ProdID'] != 'D'), ['ProdID']].drop_duplicates())
    tmp = (df.loc[(df['CustomerID'] == customer), ['ProdID']].drop_duplicates())
    val1 = len(tmp)
    val2 = len(tmp.loc[tmp['ProdID'] != 'D'])
    if val1 != val2:
        custlist.append(customer)
custlist

[14527,
 16029,
 13408,
 13694,
 14911,
 14031,
 15299,
 16013,
 16422,
 17450,
 15838,
 15498,
 17340,
 16684,
 14646,
 13137,
 17940,
 15796,
 12901,
 16672,
 13316,
 14912,
 12830,
 18167]

[14527,
 16029,
 13408,
 13694,
 14911,
 14031,
 15299,
 16013,
 16422,
 17450,
 15838,
 15498,
 17340,
 16684,
 14646,
 13137,
 17940,
 15796,
 12901,
 16672,
 13316,
 14912,
 12830,
 18167]

In [40]:
unique_products = []
unique_products_ret = []
total_products = []
total_sale = []
total_sale_wret = []
total_prod_returned = []
min_prod_spent = []
max_prod_spent = []
total_orders = []
mean_prod_sale = []

#Discount shouldn't be part of products bought nor returned 
for customer in tqdm(df['CustomerID'].unique(), total=len(data), desc="Iterating over customers"):
    # use set ?
    # dividing filters instead of conjuncting them is MUCH FASTER
    df_temp = df.loc[df['CustomerID'] == customer, ['ProdID']].drop_duplicates()
    unique_products.append(len(df_temp.loc[df_temp['ProdID'] != 'D']))
    
    df_temp = df.loc[(df['CustomerID'] == customer) & (df['Qta'] < 0), ['ProdID']].drop_duplicates()
    unique_products_ret.append(len(df_temp.loc[df_temp['ProdID'] != 'D']))
    
    
    # discount are counted in total orders
    total_orders.append(len(df.loc[(df['CustomerID'] == customer) & (df['Qta'] > 0), ['BasketID']].drop_duplicates()))
    
    
    # discount values are useful in meanps and tswr
    # discount values are NOT useful in mps, since they would always be the min value
    # discount values are useful in ts but have to be avoided in tp (not a real product)
    df_aux = df.loc[df['CustomerID'] == customer, ['Qta', 'Sale','ProdID']]
    tp = tpr = ts = tswr = 0
    mps = []
    meanps = []
    for _, r in df_aux.iterrows():
        cost = r['Qta']*r['Sale']
        tp += r['Qta'] if r['ProdID'] != 'D' else 0
        ts += cost
        if r['Qta'] > 0 or r['ProdID'] == 'D':
            tswr += cost
            meanps.append(r['Sale'])
            if r['Qta'] > 0: # not DISCOUNT
                mps.append(cost)
        else:
            tpr += 1 # or -r['Qta']
            
    total_products.append(tp)
    total_prod_returned.append(tpr)
    total_sale.append(ts)
    total_sale_wret.append(tswr)
    min_prod_spent.append(min(mps))
    max_prod_spent.append(max(mps))
    mean_prod_sale.append(stat.fmean(meanps)) 

In [39]:
order_stats_per_customer = {} # key = customerID, value = list(max_products_order, min, mean_accum, max_spent_order, min, mean_accum, mean_counter) 
mean_counter = 0
data_about_order = df[['BasketID','CustomerID']].drop_duplicates()

######################################################################################################################################
#what about discount? no sense reasoning about those records if calculating max/min qta/sale, but reasonable to add them in mean calc#
######################################################################################################################################

for _, r in tqdm(data_about_order.iterrows(), total=len(data_about_order), desc="Iterating over orders"):
    customer = r["CustomerID"]
    order = r["BasketID"]
    df_aux = df.loc[(df['BasketID'] == order) & (df['CustomerID'] == customer), ['Qta','Sale','ProdID']]  
    temp_prod = 0
    temp_sale = 0 
    
    for _, int_r in df_aux.iterrows():
        temp_prod += int_r['Qta']
        temp_sale += int_r['Qta']*int_r['Sale']
        
    is_neg = False
    if temp_prod < 0 or temp_sale < 0:
        is_neg = True  
    
    if customer not in order_stats_per_customer:
        order_stats_per_customer[customer] = [math.inf, math.inf, 0, math.inf, math.inf, 0, 0]
        
    if not is_neg:
        if order_stats_per_customer[customer][0] == math.inf or order_stats_per_customer[customer][0] < temp_prod:
            order_stats_per_customer[customer][0] = temp_prod
        if order_stats_per_customer[customer][1] == math.inf or order_stats_per_customer[customer][1] > temp_prod:
            order_stats_per_customer[customer][1] = temp_prod
        if order_stats_per_customer[customer][3] == math.inf or order_stats_per_customer[customer][3] < temp_sale:
            order_stats_per_customer[customer][3] = temp_sale
        if order_stats_per_customer[customer][4] == math.inf or order_stats_per_customer[customer][4] > temp_sale:
            order_stats_per_customer[customer][4] = temp_sale
            
    order_stats_per_customer[customer][2] += temp_prod
    order_stats_per_customer[customer][5] += temp_sale
    order_stats_per_customer[customer][6] += 1

In [41]:
max_spent_order_wise = []
min_spent_order_wise = []
mean_spent_order_wise = []
max_products_order_wise = []
min_products_order_wise = []
mean_products_order_wise = []

for customer in tqdm(df['CustomerID'].unique(), total=len(data), desc="Iterating over customers"):
    max_products_order_wise.append(order_stats_per_customer[customer][0])
    min_products_order_wise.append(order_stats_per_customer[customer][1])
    mean_products_order_wise.append(order_stats_per_customer[customer][2] / order_stats_per_customer[customer][6])
    max_spent_order_wise.append(order_stats_per_customer[customer][3])
    min_spent_order_wise.append(order_stats_per_customer[customer][4])
    mean_spent_order_wise.append(order_stats_per_customer[customer][5] / order_stats_per_customer[customer][6])

In [42]:
mean_order_month = []
mean_prod_month = []
mean_sale_month = []
for customer in tqdm(df['CustomerID'].unique(), total=len(data), desc="Iterating over customers"):
    order_per_month = []
    prod_per_month = []
    sale_per_month = []
    df_aux = df.loc[df['CustomerID'] == customer, ['BasketDate', 'BasketID', 'ProdID', 'Sale', 'Qta']]
    for month in range(1, 13):
        tot = 0
        orders = set()
        prods = set()
        for _, r in df_aux.iterrows():
             if r['BasketDate'][3:5] == str(month):
                orders.add(r['BasketID'])
                prods.add(r['ProdID'])
                tot += r['Qta']*r['Sale']
        order_per_month.append(len(orders))
        prod_per_month.append(len(prods))
        sale_per_month.append(tot)
    mean_order_month.append(stat.fmean(order_per_month))
    mean_prod_month.append(stat.fmean(prod_per_month))
    mean_sale_month.append(stat.fmean(sale_per_month))

In [43]:
# I: the total number of items purchased by a customer during the period of observation.
data['TProd'] = total_products

In [44]:
# Iu: the number of distinct items bought by a customer in the period of observation.
data['DProd'] = unique_products

In [45]:
# Imax: the maximum number of items purchased by a customer during a shopping session
data['MaxPO'] = max_products_order_wise

In [46]:
# Imin: the minimum number of items purchased by a customer during a shopping session
data['MinPO'] = min_products_order_wise

In [ ]:
# E: the Shannon entropy on the purchasing behaviour of the customer

In [47]:
# ProdPerOrderMean: mean number of items purchased by a customer during a shopping session
data['MeanProdOrder'] = mean_products_order_wise

In [48]:
# SaleTot: total amount spent during the period of observation
data['TSale'] = total_sale

In [49]:
# SaleTotWithoutReturn: total amount spent without negative quantity during the period of observation
data['TSaleWRet'] = total_sale_wret

In [50]:
# SaleMin: min amount spent for a product during the shopping session
data['MinPSale'] = min_prod_spent

In [51]:
# SaleMax: max amount spent for a product during the shopping session
data['MaxPSale'] = max_prod_spent

In [52]:
# SaleMeanPerOrder: mean amount spent for each order during the period of observation
data['MeanSaleOrder'] = mean_spent_order_wise

In [53]:
# NumRetProd: number of returned products in the period of observation
data['TRProd'] = total_prod_returned

In [54]:
# NumDistRetProd: number of distinct returned products in the period of observation
data['TRDProd'] = unique_products_ret

In [55]:
# MeanProdCostInOrder: mean cost of the products in the orders in the period of observation
data['MeanPSale'] = mean_prod_sale

In [56]:
# NumberOfOrders: total number of orders made by customer
data['TOrder'] = total_orders

In [57]:
# MeanOrderPerMonth
data['OrderMonth'] = mean_order_month

In [58]:
# MeanProductPerMonth
data['ProdMonth'] = mean_prod_month

In [59]:
# MeanAmountSpentPerMonth
data['SaleMonth'] = mean_sale_month

In [ ]:
# CustomerCounty : value from original dataframe
#data['CustomerCountry'] = ''
#data = data.join(pd.core.frame.DataFrame({'CustomerID':df['CustomerID'], 'CustomerCountry':df['CustomerCountry']}).drop_duplicates().set_index('CustomerID'), on='CustomerID')

#tmp = pd.core.frame.DataFrame({'CustomerID':df['CustomerID'], 'CustomerCountry':df['CustomerCountry']}).drop_duplicates()
#tmp['CustomerID'].value_counts()[0:8]
# 4372 distinct customers - 4380 elements in tmp
# 8 customers have two different nationalities

# Find entries with multiple nationality
#counts = tmp['CustomerID'].value_counts().items()
#count_dict = {x[0]:x[1] for x in counts}
#for _, row in tmp.sort_values(by=['CustomerID']).iterrows():
#    if count_dict[row['CustomerID']] > 1:
#        print(f"{row['CustomerID']}-{row['CustomerCountry']}")

In [60]:
data

,CustomerID,TProd,DProd,MaxPO,MinPO,MeanProdOrder,TSale,TSaleWRet,MinPSale,MaxPSale,MeanSaleOrder,TRProd,TRDProd,MeanPSale,TOrder,OrderMonth,ProdMonth,SaleMonth
0,17850,1702,21,141,12,48.628571,5317.89,5391.21,6.36,107.25,151.939714,8,6,3.960370,34,2.833333,1.750000,449.267500
1,13047,1356,106,309,1,79.764706,3094.05,3237.54,4.95,68.00,182.002941,23,20,3.932035,10,0.500000,4.083333,75.105000
2,12583,5009,115,716,101,278.277778,7187.34,7281.38,6.80,132.80,399.296667,4,4,3.103603,15,0.500000,6.500000,248.013333
3,13748,439,24,208,8,87.800000,948.25,948.25,9.36,204.00,189.650000,0,0,3.996429,5,0.083333,0.083333,17.000000
4,15100,58,1,32,16,9.666667,635.10,876.00,175.20,350.40,105.850000,3,1,10.950000,3,0.333333,0.083333,41.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,13436,76,12,76,76,76.000000,196.89,196.89,10.20,25.50,196.890000,0,0,5.830000,1,0.083333,1.000000,16.407500
4335,15520,314,18,314,314,314.000000,343.50,343.50,9.48,34.68,343.500000,0,0,1.724444,1,0.083333,1.500000,28.625000
4336,13298,96,2,96,96,96.000000,360.00,360.00,90.00,270.00,360.000000,0,0,3.750000,1,0.083333,0.166667,30.000000
4337,14569,79,10,79,79,79.000000,227.39,227.39,1.65,122.40,227.390000,0,0,3.920000,1,0.083333,0.833333,18.949167


,CustomerID,TProd,DProd,MaxPO,MinPO,MeanProdOrder,TSale,TSaleWRet,MinPSale,MaxPSale,MeanSaleOrder,TRProd,TRDProd,MeanPSale,TOrder,OrderMonth,ProdMonth,SaleMonth
0,17850,1702,21,141,12,48.628571,5317.89,5391.21,6.36,107.25,151.939714,8,6,3.960370,34,2.833333,1.750000,449.267500
1,13047,1356,106,309,1,79.764706,3094.05,3237.54,4.95,68.00,182.002941,23,20,3.932035,10,0.500000,4.083333,75.105000
2,12583,5009,115,716,101,278.277778,7187.34,7281.38,6.80,132.80,399.296667,4,4,3.103603,15,0.500000,6.500000,248.013333
3,13748,439,24,208,8,87.800000,948.25,948.25,9.36,204.00,189.650000,0,0,3.996429,5,0.083333,0.083333,17.000000
4,15100,58,1,32,16,9.666667,635.10,876.00,175.20,350.40,105.850000,3,1,10.950000,3,0.333333,0.083333,41.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334,13436,76,12,76,76,76.000000,196.89,196.89,10.20,25.50,196.890000,0,0,5.830000,1,0.083333,1.000000,16.407500
4335,15520,314,18,314,314,314.000000,343.50,343.50,9.48,34.68,343.500000,0,0,1.724444,1,0.083333,1.500000,28.625000
4336,13298,96,2,96,96,96.000000,360.00,360.00,90.00,270.00,360.000000,0,0,3.750000,1,0.083333,0.166667,30.000000
4337,14569,79,10,79,79,79.000000,227.39,227.39,1.65,122.40,227.390000,0,0,3.920000,1,0.083333,0.833333,18.949167


In [ ]:
data.info()

In [ ]:
f, ax = plt.subplots(figsize=(15, 13))
correlation = data.corr()
sns.heatmap(correlation, cmap="coolwarm", vmin=0, vmax=1, annot=True, mask=np.zeros_like(correlation, dtype=np.bool), square=True, ax=ax, edgecolor='black')
plt.xticks(rotation=315)
plt.title("Correlation matrix")